In [6]:
import pandas as pd
import os

In [9]:
path = './Data/'

finame = os.path.join(path,'GOOG.csv')
print(finame)

./Data/GOOG.csv


In [90]:
df = pd.read_csv(finame,
                index_col= 'Date' ,
                parse_dates=True)
print(df.head())

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2017-06-30  926.049988  926.049988  908.309998  908.729980  908.729980   
2017-07-03  912.179993  913.940002  894.789978  898.700012  898.700012   
2017-07-05  901.760010  914.510010  898.500000  911.710022  911.710022   
2017-07-06  904.119995  914.943970  899.700012  906.690002  906.690002   
2017-07-07  908.849976  921.539978  908.849976  918.590027  918.590027   

             Volume  
Date                 
2017-06-30  2090200  
2017-07-03  1709800  
2017-07-05  1813900  
2017-07-06  1424500  
2017-07-07  1637800  


In [91]:
df['HL_PCT'] = (df['High'] - df['Close'])/ df['Close'] * 100
df['PCT_change'] = (df['Close'] - df['Open'])/ df['Open'] * 100

In [92]:
print(df.columns.values)

['Open' 'High' 'Low' 'Close' 'Adj Close' 'Volume' 'HL_PCT' 'PCT_change']


In [93]:
df = df[['Close','HL_PCT','PCT_change','Volume']]

In [94]:
print(df.head())

                 Close    HL_PCT  PCT_change   Volume
Date                                                 
2017-06-30  908.729980  1.905958   -1.870310  2090200
2017-07-03  898.700012  1.695782   -1.477776  1709800
2017-07-05  911.710022  0.307114    1.103399  1813900
2017-07-06  906.690002  0.910341    0.284255  1424500
2017-07-07  918.590027  0.321139    1.071690  1637800


In [95]:
# forecast column

forecast = 'Close'
df.fillna(-99999, inplace = True)

In [96]:
import math

forecast_out = int(math.ceil(0.01*len(df)))
print(forecast_out)

3


In [97]:
df['label'] = df[forecast].shift(-forecast_out)

In [98]:
df.dropna(inplace=True)
print(df.head())

                 Close    HL_PCT  PCT_change   Volume       label
Date                                                             
2017-06-30  908.729980  1.905958   -1.870310  2090200  906.690002
2017-07-03  898.700012  1.695782   -1.477776  1709800  918.590027
2017-07-05  911.710022  0.307114    1.103399  1813900  928.799988
2017-07-06  906.690002  0.910341    0.284255  1424500  930.090027
2017-07-07  918.590027  0.321139    1.071690  1637800  943.830017


In [43]:
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

In [99]:
X = np.array(df.drop(['label'],1))

In [100]:
X = preprocessing.scale(X)
#print(X)
X_lately = X[-forecast_out:]
#print(X_lately)
X = X[:-forecast_out:]
print(X)

[[-1.57589226  1.24261104 -1.57955272  0.76457966]
 [-1.6834987   1.00241564 -1.26137499  0.21283523]
 [-1.5439209  -0.58459588  0.83085885  0.36382523]
 ...
 [ 1.55929495 -0.16858188 -0.13568651 -0.25159317]
 [ 1.63235662 -0.62994397  0.12145489 -1.00407743]
 [ 1.56637612  0.55190716 -0.49162932 -0.51847271]]


In [103]:
df.dropna(inplace=True)
y = np.array(df['label'])

In [104]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)

ValueError: Found input variables with inconsistent numbers of samples: [288, 291]

In [82]:
clf = LinearRegression(n_jobs=-1)
clf.fit(x_train,y_train)

accuracy = clf.score(x_test,y_test)
forecast_set = clf.predict(X_lately)
print(accuracy)

0.9086820824555812


In [89]:
import datetime
df['forecast'] = np.nan

last_date = df.iloc[-1].name
print(last_date)
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

257


AttributeError: 'numpy.int64' object has no attribute 'timestamp'

In [50]:
clf1 = svm.SVR(kernel='poly')
clf1.fit(x_train,y_train)

accuracy = clf1.score(x_test,y_test)
print(accuracy)

0.1990128588964788
